In [ ]:
from pyspark.sql.functions import col, to_date

bronze_path = "/mnt/datalake/bronze/"
silver_path = "/mnt/datalake/silver/"

clientes_df = spark.read.format("delta").load(f"{bronze_path}clientes")
produtos_df = spark.read.format("delta").load(f"{bronze_path}produtos")
vendedores_df = spark.read.format("delta").load(f"{bronze_path}vendedores")
pedidos_df = spark.read.format("delta").load(f"{bronze_path}pedidos")
itens_pedido_df = spark.read.format("delta").load(f"{bronze_path}itens_pedido")

print("Tabelas da camada Bronze carregadas.")

pedidos_df_clean = pedidos_df.withColumn("data_pedido", to_date(col("data_pedido"), "yyyy-MM-dd"))
produtos_df_clean = produtos_df.withColumn("preco", col("preco").cast("double"))
itens_pedido_df_clean = itens_pedido_df.withColumn("valor_total_item", col("quantidade") * col("preco_unitario"))

print("Limpeza e transformações básicas concluídas.")

print("Iniciando a criação da tabela fato de vendas...")

fato_vendas_df = itens_pedido_df_clean.alias("it") \
    .join(pedidos_df_clean.alias("pe"), col("it.id_pedido") == col("pe.id_pedido"), "inner") \
    .join(produtos_df_clean.alias("pr"), col("it.id_produto") == col("pr.id_produto"), "inner") \
    .join(clientes_df.alias("cl"), col("pe.id_cliente") == col("cl.id_cliente"), "inner") \
    .join(vendedores_df.alias("ve"), col("pe.id_vendedor") == col("ve.id_vendedor"), "inner") \
    .select(
        col("pe.id_pedido"),
        col("pe.data_pedido"),
        col("it.id_item"),
        col("cl.id_cliente"),
        col("cl.nome_cliente"),
        col("cl.cidade"),
        col("cl.estado"),
        col("pr.id_produto"),
        col("pr.nome_produto"),
        col("pr.categoria"),
        col("ve.id_vendedor"),
        col("ve.nome_vendedor"),
        col("ve.equipe"),
        col("it.quantidade"),
        col("it.preco_unitario"),
        col("it.valor_total_item")
    )

print("Tabela fato criada com sucesso.")
display(fato_vendas_df.limit(10))

fato_vendas_df.write.format("delta") \
              .mode("overwrite") \
              .option("overwriteSchema", "true") \
              .save(f"{silver_path}fato_vendas")

clientes_df.write.format("delta").mode("overwrite").save(f"{silver_path}dim_clientes")
produtos_df_clean.write.format("delta").mode("overwrite").save(f"{silver_path}dim_produtos")
vendedores_df.write.format("delta").mode("overwrite").save(f"{silver_path}dim_vendedores")

print("\nProcessamento da camada Silver concluído!")